# **Binomial Option Pricing Model**

### **Import Packages**

In [164]:
import numpy as np

### **Set Parameters**

In [165]:
# Set Parameters
S0 = 100    # Current stock price
K  = 105    # Strike price
T  = 3/12   # Time to maturity in years
N  = 1      # Number of steps
r  = 0.04   # Annual risk free rate

u  = 1.1                # Up factor
d  = np.round(1/u,1)    # Down factor

dt = T / N                           # delta T
p = (np.exp(r * dt) - d) / (u - d)   # Risk-neutral probability
q = 1 - p                            # Risk-neutral probability

### **Binomial Trees**

##### **Stock Price Paths**

In [166]:
# Create N x N Matrix to hold stock prices
stock_price = np.zeros([N+1,N+1])
stock_price

array([[0., 0.],
       [0., 0.]])

In [167]:
# Fill the matrix with stock prices
for col in range(len(stock_price)):
    for row in range(col+1):
        stock_price[row, col] = S0 * (u**(col-row)) * (d**row)
        
stock_price

array([[100., 110.],
       [  0.,  90.]])

#### **Option Value Paths**

In [168]:
# Create N x N Matrix to hold option values
option_payoff = np.zeros([N + 1, N + 1])
option_payoff

array([[0., 0.],
       [0., 0.]])

In [169]:
# Determine option's payoff at the end of the nodes
option_payoff[:, N] = np.maximum(np.zeros(N + 1), (stock_price[:, N] - K))
option_payoff

array([[0., 5.],
       [0., 0.]])

In [170]:
# Determine option's payoff using backward induction, discounted by risk-free rate
for col in range(N - 1, -1, -1):
    for row in range(0, col + 1):
        option_payoff[row, col] = (
            np.exp(-r * dt) * (p * option_payoff[row, col + 1] + q * option_payoff[row + 1, col + 1])
        )
option_payoff

array([[2.72387874, 5.        ],
       [0.        , 0.        ]])

In [171]:
# Option price
option_payoff[0,0].round(3)

2.724

---

# **Optimized Method**

In this optimized version, we leverage NumPy's broadcasting to create the stock price matrix and the initial option payoff matrix without explicit loops. The backward induction step is also optimized using slicing to avoid unnecessary calculations.

This version should be slightly more efficient, especially for larger values of N, as it reduces the number of explicit loops and takes advantage of NumPy's optimized operations.

In [172]:
# Create N x N matrix to hold stock prices and option payoff
stock_price = np.zeros([N+1, N+1])
option_payoff = np.zeros([N+1, N+1])

In [173]:
# Calculate stock prices and at the end of the nodes
stock_price[:, N] = S0 * (u**np.arange(N, -1, -1)) * (d**np.arange(N + 1))
stock_price

array([[  0., 110.],
       [  0.,  90.]])

In [174]:
# Calculate option payoff at the end of the the nodes
option_payoff[:, N] = np.maximum(np.zeros(N + 1), (stock_price[:, N] - K))
option_payoff

array([[0., 5.],
       [0., 0.]])

In [175]:
# Calculate stock price and option_payoff using backward induction
for col in range(N - 1, -1, -1):
    # 0:col+1 = row
    stock_price[0:col+1, col] = stock_price[0:col+1, col+1] / u
    
    option_payoff[0:col+1, col] = (
        np.exp(-r * dt) * ((p * option_payoff[0:col+1, col + 1]) + (q * option_payoff[1:col+2, col + 1]))
    )

In [176]:
# Stock Price Paths
stock_price

array([[100., 110.],
       [  0.,  90.]])

In [177]:
# Option Payoff Paths
option_payoff

array([[2.72387874, 5.        ],
       [0.        , 0.        ]])

In [178]:
# Option Price
option_payoff[0,0].round(3)

2.724

---

Both methods yield the same result for the option price, but the second method is more efficient and faster for larger numbers of steps, such as 5000 steps. The reason is that the second method utilizes vectorized operations provided by NumPy, eliminating the need for nested loops. Vectorization allows the calculations to be performed on entire arrays at once, making it more computationally efficient compared to explicitly looping through individual elements.

In summary, the second method is preferable, especially when dealing with a large number of steps, as it takes advantage of NumPy's optimized operations and offers better performance.